### Tools Implementation

react agent can calculate area, perimeter, and diagonal of a rectangle. It can also calculate the area of a circle given the radius.

tools needed:
1. input: shape of object. output: formula needed
2. Calculate result using formula. Input: substituted expression, query, output: result

In [18]:
# area, perimeter, diagonal and for all 2D shapes

import math

def determine_formula(shape: str, property: str):
    formulae = {
        "rectangle": {
            "area": "length * breadth",
            "perimeter": "2 * (length + breadth)",
            "diagonal": "sqrt(length ** 2 + breadth ** 2)"
        },
        "square": {
            "area": "side ** 2",
            "perimeter": "4 * side",
            "diagonal": "sqrt(2) * side"
        },
        "circle": {
            "area": "pi * radius ** 2",
            "perimeter": "2 * pi * radius",
            "diameter": "2 * radius",
            "circumference": "2 * pi * radius"
        },
        "triangle": {
            "area": "0.5 * base * height",
            "perimeter": "side1 + side2 + side3",
            "diagonal": "sqrt(side1 ** 2 + side2 ** 2)"
        },
        "parallelogram": {
            "area": "base * height",
            "perimeter": "2 * (side1 + side2)",
            "diagonal": "sqrt(side1 ** 2 + side2 ** 2 + 2 * side1 * side2 * cos(angle))"
        },
    }

    if shape in formulae:
        if property in formulae[shape]:
            return formulae[shape][property]
        else:
            return "Invalid formula"
        
    else:
        return "Invalid shape"
    
def calculate_expression(expression: str):
    return eval(expression, {"__builtins__": None}, {"sqrt": math.sqrt, "pi": math.pi, "cos": math.cos})


tools_description = {
    "determine_properties": {
        "description": "Always use this tool to determine the formula required.",
        "input_params": {
            "shape": "The shape for which the formula is to be determined",
            "property": "The property for which the formula needs to determined"
        }
    },
    "calculate_expression": {
        "description": "Always use this tool to provide reliable answers for expressions.",
        "input_params": {
            "expression": "The expression to be calculated"
        }
    }
}

tools_available={
    "determine_properties": determine_formula,
    "calculate_expression": calculate_expression
}

#### ReAct Agent Implementation

In [19]:
# ReAct Prompt
PROMPT_TEMPLATE = """
You are an AI assistant capable of reasoning and acting to answer complex questions. Your task is to use the tools provide and the steps already taken to the answer the given question.

# You have access to the following tools:
{tools_desc}

# Steps so far:
{steps}

# To solve the problem, follow these steps:
1. Analyse the question and use the ONE tool which could help you the most to answer the question. 
2. Always use a tool whenever possible to get reliable answers.
3. Observe whether the response the of the tool is enough to answer the question.
4. If not enough, use the another tools that can help you to answer the question.
5. If one tool does not provide the required information, then use try using another tool.
6. Repeat the process until you have enough information to answer the question.
7. Once you have the final answer, respond ONLY with the answer.


Show your reasoning and actions for each step. Use the format:
Thought: your thought process on the question
Action: the tool to be used that can help you to answer the question and its input

Answer the following question:
{question}

{{
    "is_done": true or false, whether the questions has been satisfactorily answered,
    "thought": 'thought process',
    "action": {{'tool_name': '', tool_input: {{}} }},
    "answer": "provide only this key in the response to respond with final answer"
}}
"""

from openai import OpenAI
from dotenv import load_dotenv
import json

load_dotenv("../../../.env")


class ReActAgent:
    def __init__(self, tools_available, tools_desc: dict, model: str, verbose:bool = True):
        self.tools_available = tools_available
        self.tools_desc = tools_desc
        self.model = model
        self.client = OpenAI()
        self.verbose = verbose

    def complete(self, question, steps) -> dict:
        prompt = PROMPT_TEMPLATE.format(
            tools_desc=self.tools_desc,
            steps=steps,
            question=question
        )

        response = self.client.chat.completions.create(
            model=self.model,
            response_format={ "type": "json_object" },
            messages=[
                {"role": "system", "content": "You are a helpful asssistant who responds in JSON"},
                {"role": "user", "content": prompt}
            ]
        )

        return json.loads(response.choices[0].message.content)
    
    def query(self, question):
        step_count = 0
        steps = []
        final_response = ""

        while True:
            response = self.complete(question, steps)
            step_count += 1

            if response["is_done"]:
                final_response = response["answer"]
                break
            
            tool_name, tool_input = response["action"]["tool_name"], response["action"]["tool_input"]
            tool_output = self.tools_available[tool_name](**tool_input)
            step = f"""
            Thought: {response["thought"]}
            Action: {tool_name}({tool_input})
            Observation: {tool_output}
            """
            print(step) if self.verbose else None
            steps.append(step)
        

        return {
            "content": final_response,
            "steps": steps
        }

In [20]:
questions = [
    "Calculate the area of rectangle with length 5 cm and breadth 12 cm",
    "what is the area of circle with radius 10.35 inches",
    "provide the area and perimeter of a square with sides 23m",
    "what is the perimeter of a triangle with sides 10, 12 and 15 units",
    "calculate the diagonal of a rectangle with length 10 cm and breadth 12 cm",
    "what is the area of a parallelogram with base 12 cm, height 5 cm and angle 30 degrees"
]

agent = ReActAgent(
    tools_available=tools_available,
    tools_desc=tools_description,
    model="gpt-4o-mini",
    verbose=True
)

question = questions[2]
print(f"Question: {question}")
response = agent.query(question)
print(response["content"])

Question: provide the area and perimeter of a square with sides 23m

            Thought: To find the area and perimeter of a square with side length of 23m, I need to determine the formulas for both the area and perimeter. The shape is a square and the properties needed are area and perimeter.
            Action: determine_properties({'shape': 'square', 'property': 'area'})
            Observation: side ** 2
            

            Thought: I have already determined that the formula for the area of a square is side ** 2. Now, I need to determine the formula for the perimeter of the square as well. The property needed is the perimeter.
            Action: determine_properties({'shape': 'square', 'property': 'perimeter'})
            Observation: 4 * side
            

            Thought: I have determined the formulas for the area and perimeter of a square. The side length of the square is 23m. I need to calculate the area using the formula side ** 2 and the perimeter using the form